#Reddit API (McDonald)

In [ ]:
!pip install praw pandas

In [ ]:
import praw
import pandas as pd
import datetime
import time
import requests
import unicodedata
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from google.colab import files

In [ ]:
# Reddit API credentials
reddit = praw.Reddit(
    client_id="zDKZOEaOZUmPEzxji5j2cw",
    client_secret="rWdUyUbQRITUjsYnFkAbYNHCT4ntnw",
    user_agent="Phupha Data Scraper v1.0 by /u/AmbitionNorth7402",
    username="AmbitionNorth7402",
    password="Satang0982645021"
)

print(f"Logged in as: {reddit.user.me()}")

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



Logged in as: AmbitionNorth7402


In [ ]:
subreddit_name = "mcdonalds"

# Set time range
start_date = datetime.datetime(2024, 3, 1)
end_date = datetime.datetime(2025, 3, 1)
start_timestamp = start_date.timestamp()
end_timestamp = end_date.timestamp()

# Initialise variables
comments_list = []
seen_comments = set()


def clean_text(text):
    """Normalizes Unicode characters and removes encoding errors."""
    if isinstance(text, str):
        text = text.encode("utf-8", "ignore").decode("utf-8")
        text = unicodedata.normalize("NFKD", text)
        text = text.replace("\u2019", "'")
        text = text.replace("\u2013", "-")
        text = text.replace("\u2014", "--")
        text = text.strip()
    return text


def safe_request(submission):
    """Fetch comments while handling rate limits (HTTP 429 errors)."""
    while True:
        try:
            submission.comments.replace_more(limit=None)
            return submission.comments.list()
        except requests.exceptions.RequestException as e:
            if "429" in str(e):
                print("Rate limit reached. Waiting for 60 seconds before retrying...")
                time.sleep(60)
            else:
                raise e

subreddit = reddit.subreddit(subreddit_name)
posts = list(subreddit.hot(limit=None)) + list(subreddit.new(limit=None)) + list(subreddit.top(limit=None))

for post in posts:
    submission = reddit.submission(id=post.id)

    time.sleep(2)

    comments = safe_request(submission)

    for comment in comments:
        if comment.body not in ["[removed]", "[deleted]"]:
            comment_timestamp = comment.created_utc


            if start_timestamp <= comment_timestamp <= end_timestamp:
                timestamp = datetime.datetime.fromtimestamp(comment_timestamp)


                cleaned_title = clean_text(post.title)
                cleaned_comment = clean_text(comment.body)


                unique_key = (cleaned_title, cleaned_comment)
                if unique_key not in seen_comments:
                    seen_comments.add(unique_key)
                    comments_list.append([cleaned_title, cleaned_comment, comment.score, timestamp])


df_comments = pd.DataFrame(comments_list, columns=["Post Title", "Comment", "Upvotes", "Timestamp"])


output_filename = f"{subreddit_name}_comments_2024_2025.csv"
df_comments.to_csv(output_filename, index=False, encoding="utf-8-sig")

print(f"Collected {len(df_comments)} unique comments from r/{subreddit_name} between {start_date.date()} and {end_date.date()}.")
df_comments.head()


Streaming output truncated to the last 5000 lines.
It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.r

Collected 5253 unique comments from r/mcdonalds between 2024-03-01 and 2025-03-01.


,Post Title,Comment,Upvotes,Timestamp
0,E. coli Outbreak Linked to McDonald's Quarter ...,Update:\n\n[E. coli outbreak linked to McDonal...,1,2024-10-22 22:39:15
1,E. coli Outbreak Linked to McDonald's Quarter ...,"I just had it 3 hrs ago in northeast. In app, ...",19,2024-10-23 02:47:40
2,E. coli Outbreak Linked to McDonald's Quarter ...,So many food outbreaks lately,28,2024-10-22 23:31:41
3,E. coli Outbreak Linked to McDonald's Quarter ...,My husband was in the ER yesterday likely from...,6,2024-10-24 05:48:06
4,E. coli Outbreak Linked to McDonald's Quarter ...,I ate a quarter Pounder today while scrolling ...,8,2024-10-23 00:29:59


In [ ]:
# Load the dataset
df = pd.read_csv("mcdonalds_comments_2024_2025.csv")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5253 entries, 0 to 5252
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Post Title  5253 non-null   object
 1   Comment     5253 non-null   object
 2   Upvotes     5253 non-null   int64 
 3   Timestamp   5253 non-null   object
dtypes: int64(1), object(3)
memory usage: 164.3+ KB


In [ ]:
df["Timestamp"] = pd.to_datetime(df["Timestamp"])

# Check the period of this dataset
start_date = df["Timestamp"].min()
end_date = df["Timestamp"].max()

start_date, end_date

(Timestamp('2024-03-01 00:40:00'), Timestamp('2025-02-28 23:23:00'))